<a href="https://colab.research.google.com/github/Demiarge/kaggle-platesv2/blob/main/kaggle_compi_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle input and unzipping data to use properly

In [2]:
# install Kagglee
!pip install kaggle

# Upload Kaggle API key
from google.colab import files
files.upload()

# Move API key to the app location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the competition data
!kaggle competitions download -c platesv2

# Unzip the data
!unzip platesv2.zip -d /content/temp
!rm -rf platesv2.zip #dlt

# Unzip main data
!cd /content/temp/
!ls
!unzip /content/temp/plates.zip -d /content/platesv2
!rm -rf /content/temp/

# move data in data folder
!mv /content/platesv2/plates/ /content/data/
!rm -rf /content/platesv2/

KeyboardInterrupt: 

In [ ]:
# Additional cell for use

!ls -la

# Import library

# data traning
